<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

## Tackling an NLP Problem with Naive Bayes

----

We can sketch out the data science process as follows:
1. Define the problem.
2. Obtain the data.
3. Explore the data.
4. Model the data.
5. Evaluate the model.
6. Answer the problem.

**Instructor's note: While I completely agree with the idea expressed below, we will be discussing Naive Bayes in class**

In this lab, we are going to apply a **new** modeling technique to natural language processing data.

> "But how can we apply a modeling technique we haven't learned?!"

The DSI program is great - but we can't teach you *everything* about data science in 12 weeks! This lab is designed to help you start learning something new without it being taught in a formal lesson. 

- Later in the cohort (like for your capstone!), you'll be exploring models, libraries, and resources that you haven't been explicitly taught.
- After the program, you'll want to continue developing your skills. Being comfortable with documentation and being confident in your ability to read something new and decide whether or not it is an appropriate method for the problem you're trying to solve is **incredibly** valuable.

### Step 1: Define the problem.

Many organizations have a substantial interest in classifying users of their product into groups. Some examples:
- A company that serves as a marketplace may want to predict who is likely to purchase a certain type of product on their platform, like books, cars, or food.
- An application developer may want to identify which individuals are willing to pay money for "bonus features" or to upgrade their app.
- A social media organization may want to identify who generates the highest rate of content that later goes "viral."

### Summary
In this lab, you're an engineer for Facebook. In recent years, the organization Cambridge Analytica gained worldwide notoriety for its use of Facebook data in an attempt to sway electoral outcomes.

Cambridge Analytica, an organization staffed with lots of Ph.D. researchers, used the Big5 personality groupings (also called OCEAN) to group people into one of 32 different groups.
- The five qualities measured by this personality assessment are:
    - **O**penness
    - **C**onscientiousness
    - **E**xtroversion
    - **A**greeableness
    - **N**euroticism
- Each person could be classified as "Yes" or "No" for each of the five qualities.
- This makes for 32 different potential combinations of qualities. ($2^5 = 32$)
- You don't have to check it out, but if you want to learn more about this personality assessment, head to [**the Wikipedia page**](https://en.wikipedia.org/wiki/Big_Five_personality_traits).
- There's also [**a short (3-4 pages) academic paper describing part of this approach**](./celli-al_wcpr13.pdf).

Cambridge Analytica's methodology was, roughly, the following:
- Gather a large amount of data from Facebook.
- Use this data to predict an individual's Big5 personality "grouping."
- Design political advertisements that would be particularly effective to that particular "grouping." (For example, are certain advertisements particularly effective toward people with specific personality traits?)

You want to know the **real-world problem**: "Is what Cambridge Analytica attempted to do actually possible, or is it junk science?"

However, we'll solve the related **data science problem**: "Are one's Facebook statuses predictive of whether or not one is agreeable?"
> Note: If Facebook statuses aren't predictive of one being agreeable (one of the OCEAN qualities), then Cambridge Analytica's approach won't work very well!

### Step 2: Obtain the data.

Obviously, there are plenty of opportunities to discuss the ethics surrounding this particular issue... so let's do that.

In [16]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.pipeline import Pipeline

from datetime import datetime

In [2]:
data = pd.read_csv('./mypersonality_final.csv', encoding = 'ISO-8859-1')

In [3]:
data.head()

,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,cOPN,DATE,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,06/19/09 03:21 PM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
1,b7b7764cfa1c523e4e93ab2a79a946c4,is so sleepy it's not even funny that's she ca...,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,07/02/09 08:41 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
2,b7b7764cfa1c523e4e93ab2a79a946c4,is sore and wants the knot of muscles at the b...,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,06/15/09 01:15 PM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
3,b7b7764cfa1c523e4e93ab2a79a946c4,likes how the day sounds in this new song.,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,06/22/09 04:48 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1
4,b7b7764cfa1c523e4e93ab2a79a946c4,is home. <3,2.65,3.0,3.15,3.25,4.4,n,y,n,n,y,07/20/09 02:31 AM,180.0,14861.6,93.29,0.03,15661.0,0.49,0.1


**1. What is the difference between anonymity and confidentiality? All else held equal, which tends to keep people safer?**

In [ ]:
# Anonymity: Even collector doesn't know who answer the survey
# Confidentiality: We know the identities of people we survey, but keeping their information secret.
# Thus, Anonymity keeps the privacy of data, even it leaks

**2. Suppose that the "unique identifier" in the above data, the `#AUTHID`, is a randomly generated key so that it can never be connected back to the original poster. Have we guaranteed anonymity here? Why or why not?**

In [ ]:
# If the other columns can't imply the identities of people we survey, then it can guarantee the anonymity.

**3. As an engineer for Facebook, you recognize that user data will be used by Facebook and by other organizations - that won't change. However, what are at least three recommendations you would bring to your manager to improve how data is used and shared? Be as specific as you can.**

In [ ]:
# 1. Users should give permissions the data that users agree to share
# 2. Gathering the minimal data we want, not gather the unnecessary data. Less risks if it leaks
# 3. Feedback to users how given data is going to process and analyze

### Step 3: Explore the data.

- Note: For our $X$ variable, we will only use the `STATUS` variable. For our $Y$ variable, we will only use the `cAGR` variable.

**4. Explore the data here.**
> We aren't explicitly asking you to do specific EDA here, but what EDA would you generally do with this data? Do the EDA you usually would, especially if you know what the goal of this analysis is.

In [6]:
data[['STATUS','cAGR']].isnull().sum()

STATUS    0
cAGR      0
dtype: int64

In [7]:
data[['STATUS','cAGR']].dtypes

STATUS    object
cAGR      object
dtype: object

In [8]:
data['cAGR'].value_counts()

cAGR
y    5268
n    4649
Name: count, dtype: int64

In [ ]:
# There are no null in X and y. But we will mapping y into integer for doing analysis

**5. What is the difference between CountVectorizer and TFIDFVectorizer?**

In [ ]:
# CountVectorizer: count the number of occurences of each word in documents
# TFIDFVectorizer: apply the formula in number of occurrences: each word in documents, number of document contains this word
# Thus, TFIDFVectorizer will check how common or rare of each word.

**6. What are stopwords?**

In [ ]:
# The words having meaningless in sentence, or very common words; likes: on, where, to, I, etc.

**7. Give an example of when you might remove stopwords.**

In [ ]:
# When documents are in form of long passages.
# When the sentiment is not involved. Because removing stopwords 'not' will change the meaning

**8. Give an example of when you might keep stopwords in your model.**

In [ ]:
# When conducting sentiment analysis.

### Step 4: Model the data.

We are going to fit two types of models: a logistic regression and a [**Naive Bayes classifier**](https://scikit-learn.org/stable/modules/naive_bayes.html).

**Reminder:** We will only use the feature `STATUS` to model `cAGR`.

### We want to attempt to fit our models on sixteen sets of features:

1. CountVectorizer with 100 features, with English stopwords removed and with an `ngram_range` that includes 1 and 2.
2. CountVectorizer with 100 features, with English stopwords removed and with the default `ngram_range`.
3. CountVectorizer with 100 features, with English stopwords kept in and with an `ngram_range` that includes 1 and 2.
4. CountVectorizer with 100 features, with English stopwords kept in and with the default `ngram_range`.
5. CountVectorizer with 500 features, with English stopwords removed and with an `ngram_range` that includes 1 and 2.
6. CountVectorizer with 500 features, with English stopwords removed and with the default `ngram_range`.
7. CountVectorizer with 500 features, with English stopwords kept in and with an `ngram_range` that includes 1 and 2.
8. CountVectorizer with 500 features, with English stopwords kept in and with the default `ngram_range`.
9. TFIDFVectorizer with 100 features, with English stopwords removed and with an `ngram_range` that includes 1 and 2.
10. TFIDFVectorizer with 100 features, with English stopwords removed and with the default `ngram_range`.
11. TFIDFVectorizer with 100 features, with English stopwords kept in and with an `ngram_range` that includes 1 and 2.
12. TFIDFVectorizer with 100 features, with English stopwords kept in and with the default `ngram_range`.
13. TFIDFVectorizer with 500 features, with English stopwords removed and with an `ngram_range` that includes 1 and 2.
14. TFIDFVectorizer with 500 features, with English stopwords removed and with the default `ngram_range`.
15. TFIDFVectorizer with 500 features, with English stopwords kept in and with an `ngram_range` that includes 1 and 2.
16. TFIDFVectorizer with 500 features, with English stopwords kept in and with the default `ngram_range`.

**9. Rather than manually instantiating 16 different vectorizers, what `sklearn` class have we learned about that might make this easier? Use it.**

In [18]:
# Define X and y
X = data['STATUS']
y = data['cAGR'].map({'y':1, 'n':0})   # Let y be positive class

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.8, stratify = y, random_state = 42)

# Instantiate 2 types of vectorizer
vectorizer = [CountVectorizer(), TfidfVectorizer()]

# Instantiate Classifier as Logistic model
classifier = LogisticRegression()

# create Pipeline
pipe =  Pipeline([
        ('vectorizer', CountVectorizer())                    # Default vectorizer, will be replaced in grid search
        , ('classifier', classifier)                         # classifier is LogisticRegression we predefined         
    ])

# Set vectorizer parameters
params = {
    'vectorizer': [CountVectorizer(), TfidfVectorizer()]
    , 'vectorizer__max_features': [100, 500]                 # max feature is 100 or 500
    , 'vectorizer__stop_words': [None, 'english']            # no removing or including stopwords
    , 'vectorizer__ngram_range': [(1,1), (1,2)]              # uni-gram; uni-gram and bi-gram
}

# Instantiate grid search with parameters
gs = GridSearchCV(estimator = pipe, param_grid = params, cv = 5, scoring = 'accuracy', n_jobs = -1)

# Fit grid search
t = datetime.now()

gs.fit(X_train, y_train)

# Print statistics
print(f'Best parameters: {gs.best_params_}\n')
print(f'Best score for: {gs.best_score_}\n')
print(f'Training score for: {gs.score(X_train, y_train)}\n')
print(f'Testing score for: {gs.score(X_test, y_test)}\n')
print(f'Time usage for grid search: {datetime.now() - t}\n')

Best parameters: {'vectorizer': TfidfVectorizer(), 'vectorizer__max_features': 500, 'vectorizer__ngram_range': (1, 1), 'vectorizer__stop_words': None}

Best score for: 0.5502319841778766

Training score for: 0.6259926887684356

Testing score for: 0.5372983870967742

Time usage for grid search: 0:00:02.570343



**10. What are some of the advantages of fitting a logistic regression model?**

In [ ]:
# It is very fast!!!

**11. Fit a logistic regression model and compare it to the baseline.**

In [22]:
# We've done in Q9.
y_train.value_counts(normalize = True)   # Get baseline Train = 0.53
y_test.value_counts(normalize = True)    # Get baseline Test = 0.53

# The testing score from grid search and baseline are approximately same.
# Our model didn't do much

cAGR
1    0.53125
0    0.46875
Name: proportion, dtype: float64

### Summary of Naive Bayes 

Naive Bayes is a classification technique that relies on probability to classify observations.
- It's based on a probability rule called **Bayes' Theorem**... thus, "**Bayes**."
- It makes an assumption that isn't often met, so it's "**naive**."

Despite being a model that relies on a naive assumption, it often performs pretty well! (This is kind of like linear regression... we aren't always guaranteed homoscedastic errors in linear regression, but the model might still do a good job regardless.)
- [**Interested in the details?**](https://www.cs.unb.ca/~hzhang/publications/FLAIRS04ZhangH.pdf)


The [**sklearn documentation**](https://scikit-learn.org/stable/modules/naive_bayes.html) is here, but it can be intimidating. So, to quickly summarize the Bayes and Naive parts of the model...

#### Bayes' Theorem
If you've seen Bayes' Theorem, it relates the probability of $P(A|B)$ to $P(B|A)$. (Don't worry; we won't be doing any probability calculations by hand! However, you may want to refresh your memory on conditional probability from our earlier lessons if you forget what a conditional probability is.)

$$
\begin{eqnarray*}
\text{Bayes' Theorem: } P(A|B) &=& \frac{P(B|A)P(A)}{P(B)}
\end{eqnarray*}
$$

- Let $A$ be that someone is "agreeable," like the OCEAN category.
- Let $B$ represent the words used in their Facebook post.

$$
\begin{eqnarray*}
\text{Bayes' Theorem: } P(A|B) &=& \frac{P(B|A)P(A)}{P(B)} \\
\Rightarrow P(\text{person is agreeable}|\text{words in Facebook post}) &=& \frac{P(\text{words in Facebook post}|\text{person is agreeable})P(\text{person is agreeable})}{P(\text{words in Facebook post})}
\end{eqnarray*}
$$

We want to calculate the probability that someone is agreeable **given** the words that they used in their Facebook post! (Rather than calculating this probability by hand, this is done under the hood and we can just see the results by checking `.predict_proba()`.) However, this is exactly what our model is doing. We can (a.k.a. the model can) calculate the pieces on the right-hand side of the equation to give us a probability estimate of how likely someone is to be agreeable given their Facebook post.

#### Naive Assumption

If our goal is to estimate $P(\text{person is agreeable}|\text{words in Facebook post})$, that can be quite tricky.

---

<details><summary>Bonus: if you want to understand why that's complicated, click here.</summary>
    
- The event $\text{"words in Facebook post"}$ is a complicated event to calculate.

- If a Facebook post has 100 words in it, then the event $\text{"words in Facebook post"} = \text{"word 1 is in the Facebook post" and "word 2 is in the Facebook post" and }\ldots \text{ and "word 100 is in the Facebook post"}$.

- To calculate the joint probability of all 100 words being in the Facebook post gets complicated pretty quickly. (Refer back to the probability notes on how to calculate the joint probability of two events if you want to see more.)
</details>

---

To simplify matters, we make an assumption: **we assume that all of our features are independent of one another.**

In some contexts, this assumption might be realistic!

**12. Why would this assumption not be realistic with NLP data?**

In [ ]:
# For easy to calculate P(words in Facebook post)
# Instead of calculate the joint probability of each combinations of words;
# We assume 'Naive' for make all words independent for each other.
# That means the P(words in Facebook post) will equal to the product of probability of each word.

# By the way, for answering the Q12, this 'Naive' doesn't suit the realistic NLP.
# Because words are not fully independent. Some of them must be consecutive to get the full meanings.
# Example, New Yourk, not good

Despite this assumption not being realistic with NLP data, we still use Naive Bayes pretty frequently.
- It's a very fast modeling algorithm. (which is great especially when we have lots of features and/or lots of data!)
- It is often an excellent classifier, outperforming more complicated models.

There are three common types of Naive Bayes models: Bernoulli Naive Bayes, Multinomial Naive Bayes, and Gaussian Naive Bayes.
- How do we pick which of the three models to use? It depends on our $X$ variable.
    - Bernoulli Naive Bayes is appropriate when our features are all 0/1 variables.
        - [**Bernoulli NB Documentation**](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.BernoulliNB.html#sklearn.naive_bayes.BernoulliNB)
    - Multinomial Naive Bayes is appropriate when our features are variables that take on only positive integer counts.
        - [**Multinomial NB Documentation**](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB)
    - Gaussian Naive Bayes is appropriate when our features are Normally distributed variables. (Realistically, though, we kind of use Gaussian whenever neither Bernoulli nor Multinomial works.)
        - [**Gaussian NB Documentation**](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB)

**13. Suppose you CountVectorized your features. Which Naive Bayes model would be most appropriate to fit? Why? Fit it.**

In [26]:
# MultinomialNB will likely to fit, since CountVectorizer gives the nonnegative integer value

pipe_cvec =  Pipeline([
        ('vectorizer', CountVectorizer())
        , ('classifier', MultinomialNB())        
    ])

t = datetime.now()

pipe_cvec.fit(X_train, y_train)

print(f'Training score for: {pipe_cvec.score(X_train, y_train)}\n')
print(f'Testing score for: {pipe_cvec.score(X_test, y_test)}\n')
print(f'Time usage for this pipeline: {datetime.now() - t}\n')

Training score for: 0.8633556031766041

Testing score for: 0.6018145161290323

Time usage for this pipeline: 0:00:00.234213



**14. Suppose you TFIDFVectorized your features. Which Naive Bayes model would be most appropriate to fit? Why? Fit it.**

In [27]:
# I think MultinomialNB not only take postive integer features' values, but also take nonnegative number

pipe_tvec =  Pipeline([
        ('vectorizer', TfidfVectorizer())
        , ('classifier', MultinomialNB())        
    ])

t = datetime.now()

pipe_tvec.fit(X_train, y_train)

print(f'Training score for: {pipe_tvec.score(X_train, y_train)}\n')
print(f'Testing score for: {pipe_tvec.score(X_test, y_test)}\n')
print(f'Time usage for this pipeline: {datetime.now() - t}\n')

Training score for: 0.848859195764528

Testing score for: 0.6048387096774194

Time usage for this pipeline: 0:00:00.229209



**15. Compare the performance of your models.**

In [28]:
# With default configuration, the CountVectorizer performs better than TfidfVectorizer in training set.
# By the way, both seem to be overfit since the testing score is signigicant lower

**16. Even though we didn't explore the full extent of Cambridge Analytica's modeling, based on what we did here, how effective was their approach at using Facebook data to model agreeableness?**

In [ ]:
# From testing score, the accuracy is only 60%. Thus, the research is not reliable enough.